In [1]:
%matplotlib inline
from collections import defaultdict, Counter
import glob
import os
import json

from IPython.core.display import HTML, Image
from matplotlib_venn import venn3
import pandas as pd
import pybedtools
import pysam
import gspread
from gscripts.general import parsers
from gscripts.general import dataviz
%load_ext autoreload
import numpy as np
%autoreload 2
reload(parsers)
reload(pybedtools)
import seaborn as sns
import matplotlib
from oauth2client.client import SignedJwtAssertionCredentials
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter

import gzip
img_dir = "/home/gpratt/Dropbox/encode_integration/qc_work/"

/home/gpratt/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [11]:
stetson_qc = parsers.clipseq_metrics("/home/gpratt/projects/for_stetson/analysis/stetson_v1/", iclip=False)
stetson_qc["Fraction Collapsed"] = stetson_qc['Usable Reads'] / stetson_qc['Uniquely Mapped Reads'].astype(float)
stetson_qc["Fraction Usable"] = stetson_qc['Usable Reads'] / stetson_qc['Input Reads'].astype(float)


In [12]:
filtered_stetson_qc = stetson_qc[[ "Input Reads",
           #'Trimmed bases',
           "Reads Written",
           "repetitive_count",
           "Uniquely Mapped Reads",
           "Uniquely mapped reads %",
           "Usable Reads",
           "Fraction Collapsed",
           "Fraction Usable",
           #"spot",
           "Num Peaks",
           #"Passed QC"
          ]]



In [13]:
HTML(filtered_stetson_qc.to_html(formatters={"Input Reads" : parsers.commas,
                                     "Reads Written" : parsers.commas,
                                     "repetitive_count": parsers.commas,
                                     "Reads after Quality Filtering" : parsers.commas,
                                     "Uniquely Mapped Reads" : parsers.commas,
                                     "Usable Reads" : parsers.commas,
                                     "Num Peaks": parsers.commas
                                     } ))

,Input Reads,Reads Written,repetitive_count,Uniquely Mapped Reads,Uniquely mapped reads %,Usable Reads,Fraction Collapsed,Fraction Usable,Num Peaks
2_SG_Stetson_iCLIP_S1_R1_001,"99,555,463","78,802,893","40,051,587","43,075,184",78.94%,"553,386",0.012847,0.005559,"22,739"
4_SG_Stetson_iCLIP_S2_R1_001,"85,117,679","75,148,807","45,043,868","28,463,198",69.65%,"145,996",0.005129,0.001715,"16,823"
6_SG_Stetson_iCLIP_S3_R1_001,"95,000,723","83,356,872","49,830,895","43,940,044",88.84%,"387,314",0.008815,0.004077,"26,508"
8_SG_Stetson_iCLIP_S4_R1_001,"124,747,095","103,326,069","81,679,572","28,089,460",79.44%,"376,956",0.013420,0.003022,"19,544"
Undetermined_S0_R1_001,"27,408,143","26,150,616","3,224,302","2,252,896",9.45%,"624,248",0.277087,0.022776,"16,643"


# Append Barcode to Stetson_clip so I can better process data

In [ ]:
def append_fake_barcode(fn):
    with gzip.open(fn) as in_file:
        with gzip.open(".".join(fn.split(".")[:-2] + ["bc", "fastq", "gz"]), 'w') as out_file:
            while True:
                try:
                    name = in_file.next()
                    seq = in_file.next()
                    at = in_file.next()
                    score = in_file.next()

                    name = name[0] + "AAAAA:" + name[1:]

                    out_file.write(name)
                    out_file.write(seq)
                    out_file.write(at)
                    out_file.write(score)
                except StopIteration:
                    break

In [ ]:
fns = glob.glob("/home/gpratt/projects/for_stetson/data/*")

for fn in fns:
    append_fake_barcode(fn)